<a href="https://colab.research.google.com/github/TishweeMahjong/CS370_Group2/blob/Milestone2/CS370_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Milestone 1: Source Videos and Closed Captions

In [ ]:
!pip install pytube
!pip install youtube_transcript_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.1 MB/s eta 0:00:00


In [ ]:
import os
from pytube import YouTube
from pytube import Playlist
from youtube_transcript_api import YouTubeTranscriptApi

playlist_url = 'https://www.youtube.com/playlist?list=PLI1yx5Z0Lrv77D_g1tvF9u3FVqnrNbCRL'
total_videos = 10

# directory to store downloaded videos and captions
video_directory = 'downloaded_videos'
os.makedirs(video_directory, exist_ok=True)

def download_video(video_url, video_directory):
    try:
        yt = YouTube(video_url)
        video_stream = yt.streams.filter(progressive=True, file_extension='mp4').first()
        video_stream.download(output_path=video_directory)
        print(f"Downloaded: {yt.title}")
        return True
    except Exception as e:
        return False

def download_captions(video_id, video_directory):
    try:
        captions = YouTubeTranscriptApi.get_transcript(video_id)
        with open(os.path.join(video_directory, f"{video_id}_captions.json"), 'w', encoding='utf-8') as file:
            file.write(str(captions))
        print(f"Downloaded captions for video ID: {video_id}")
    except Exception as e:
        return False

# Fetch videos from the playlist
playlist = Playlist(playlist_url)

# Download videos and captions
downloaded_videos = 0

for video_url in playlist.video_urls:
    if downloaded_videos >= total_videos:
        break

    if download_video(video_url, video_directory):
        video_id = video_url.split('v=')[1]
        download_captions(video_id, video_directory)
        downloaded_videos += 1

print(f"Downloaded {downloaded_videos} videos and their captions to {video_directory}")

Downloaded: Vice President Kamala Harris: The 2023 60 Minutes Interview
Downloaded captions for video ID: G3Eup4mfJdA
Downloaded: Pink: The 60 Minutes Interview
Downloaded captions for video ID: 480OGItLZNo
Downloaded: Rich Paul: The 60 Minutes Interview
Downloaded captions for video ID: OA2Tj75T3fI
Downloaded: "Godfather of AI" Geoffrey Hinton: The 60 Minutes Interview
Downloaded captions for video ID: qrvK_KuIeJk
Downloaded: Gen. Mark Milley: The 60 Minutes Interview
Downloaded captions for video ID: oFVuQ0RP_As
Downloaded: Attorney General Merrick Garland: The 60 Minutes Interview
Downloaded captions for video ID: 4aPp8KX6EiU
Downloaded: Deion Sanders: The 2023 60 Minutes Interview
Downloaded captions for video ID: h8PSWeRLGXs
Downloaded: Volodymyr Zelenskyy: The 2023 60 Minutes Interview
Downloaded captions for video ID: Z8qC2tVkGeU
Downloaded: Denzel Washington | 60 Minutes Archive
Downloaded captions for video ID: Y9nM_9oBj2k
Downloaded: World Number 1 Pool Player Shane Van Boeni

## Milestone 2: Speech to Text

In [ ]:
# Dependencies

!pip install -q pytube
!pip install -q git+https://github.com/openai/whisper.git

import os, re
import torch
from pathlib import Path
from pytube import YouTube

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.1/153.1 kB 21.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import whisper
from whisper.utils import get_writer

# Directory containing the downloaded video files
video_directory = 'downloaded_videos'

# Directory to store the transcribed text files
transcription_directory = 'transcriptions'
os.makedirs(transcription_directory, exist_ok=True)

def transcribe_video(video_path, output_file):
    try:
        model = whisper.load_model("base")
        result = model.transcribe(video_path)
        transcribed_text = result["text"]

        with open(output_file, 'w', encoding='utf-8') as text_file:
            text_file.write(transcribed_text)

        print(f"Transcribed video: {video_path}")

    except Exception as e:
        print(f"Error transcribing video: {video_path}, Error: {str(e)}")

# Loop through the video files in the directory
for filename in os.listdir(video_directory):
    if filename.endswith(".mp4"):
        video_path = os.path.join(video_directory, filename)
        output_file = os.path.join(transcription_directory, f"{os.path.splitext(filename)[0]}.txt")
        transcribe_video(video_path, output_file)

print(f"Transcriptions saved in {transcription_directory}")

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 122MiB/s]


Transcribed video: downloaded_videos/World Number 1 Pool Player Shane Van Boening The 60 Minutes Interview.mp4
Transcribed video: downloaded_videos/Pink The 60 Minutes Interview.mp4
Transcribed video: downloaded_videos/Deion Sanders The 2023 60 Minutes Interview.mp4
Transcribed video: downloaded_videos/Denzel Washington  60 Minutes Archive.mp4
Transcribed video: downloaded_videos/Gen Mark Milley The 60 Minutes Interview.mp4
Transcribed video: downloaded_videos/Vice President Kamala Harris The 2023 60 Minutes Interview.mp4
Transcribed video: downloaded_videos/Rich Paul The 60 Minutes Interview.mp4
Transcribed video: downloaded_videos/Volodymyr Zelenskyy The 2023 60 Minutes Interview.mp4
Transcribed video: downloaded_videos/Attorney General Merrick Garland The 60 Minutes Interview.mp4
Transcribed video: downloaded_videos/Godfather of AI Geoffrey Hinton The 60 Minutes Interview.mp4
Transcriptions saved in transcriptions
